In [5]:
import pandas as pd

from shared_code import utils

In [2]:
azure_utils = utils.AzureUtils()
secret_client = azure_utils.initialize_key_vault()

sa_secret = azure_utils.get_key_vault_secret(secret_client, 'sa-secret')
sa_name = azure_utils.get_key_vault_secret(secret_client, 'sa-name')

azure_utils.initialize_storage_account_ad(sa_secret.value, sa_name.value)
azure_utils.initialize_data_lake(sa_name.value, sa_secret.value)

In [3]:
dim_company = azure_utils.download_parquet_blob("bronze/companies/sp500", "companies_sp500.parquet")
dim_company.head()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [ ]:
# class StockFundamentals:
#     def __init__(self, symbol):
#         self.symbol = symbol
#         self.ticker = Ticker(self.symbol)
#         self.income_statement = None
#         self.balance_sheet = None
#         self.cash_flow = None

#     def fetch_data(self):
#         self.income_statement = self.ticker.income_statement(frequency = 'q', trailing = False).sort_values('asOfDate').set_index("asOfDate")
#         self.balance_sheet = self.ticker.balance_sheet(frequency = 'q', trailing = False).sort_values('asOfDate').set_index("asOfDate")
#         self.cash_flow = self.ticker.cash_flow(frequency = 'q', trailing = False).sort_values('asOfDate').set_index("asOfDate")

# class PiotroskiScoreCalculator:
#     def __init__(self, stock):
#         self.stock = stock
#         self.date = None
#         self.prev_date = None

#     def set_dates(self, date):
#         self.date = date
#         self.prev_date = self.stock.balance_sheet.index[self.stock.balance_sheet.index.get_loc(date) - 1]

#     def calculate_roa_score(self):
#         roa = self.stock.income_statement.loc[self.date, 'NetIncome'] / self.stock.balance_sheet.loc[self.date, 'TotalAssets']
#         return 1 if roa > 0 else 0

#     def calculate_cfo_score(self):
#         cfo = self.stock.cash_flow.loc[self.date, 'OperatingCashFlow']
#         return 1 if cfo > 0 else 0

#     def calculate_delta_roa_score(self):
#         roa = self.stock.income_statement.loc[self.date, 'NetIncome'] / self.stock.balance_sheet.loc[self.date, 'TotalAssets']
#         delta_roa = roa - (self.stock.income_statement.loc[self.prev_date, 'NetIncome'] / \
#                            self.stock.balance_sheet.loc[self.prev_date, 'TotalAssets'])
#         return 1 if delta_roa > 0 else 0

#     def calculate_quality_of_earnings_score(self):
#         cfo = self.stock.cash_flow.loc[self.date, 'OperatingCashFlow']
#         return 1 if cfo > self.stock.income_statement.loc[self.date, 'NetIncome'] else 0

#     def calculate_delta_leverage_score(self):
#         delta_leverage = self.stock.balance_sheet.loc[self.date, 'LongTermDebt'] / self.stock.balance_sheet.loc[self.date, 'TotalAssets'] - \
#                          self.stock.balance_sheet.loc[self.prev_date, 'LongTermDebt'] / self.stock.balance_sheet.loc[self.prev_date, 'TotalAssets']
#         return 1 if delta_leverage < 0 else 0

#     def calculate_delta_liquidity_score(self):
#         delta_liquidity = (self.stock.balance_sheet.loc[self.date, 'CurrentAssets'] / self.stock.balance_sheet.loc[self.date, 'CurrentLiabilities']) - \
#                           (self.stock.balance_sheet.loc[self.prev_date, 'CurrentAssets'] / self.stock.balance_sheet.loc[self.prev_date, 'CurrentLiabilities'])
#         return 1 if delta_liquidity > 0 else 0

#     def calculate_new_equity_score(self):
#         new_equity = self.stock.balance_sheet.loc[self.date, 'CommonStock'] - \
#                      self.stock.balance_sheet.loc[self.prev_date, 'CommonStock']
#         return 1 if new_equity <= 0 else 0

#     def calculate_gross_margin_score(self):
#         gross_margin_now = (self.stock.income_statement.loc[self.date, 'TotalRevenue'] - \
#                             self.stock.income_statement.loc[self.date, 'CostOfRevenue']) / self.stock.income_statement.loc[self.date, 'TotalRevenue']
#         gross_margin_prev = (self.stock.income_statement.loc[self.prev_date, 'TotalRevenue'] - \
#                              self.stock.income_statement.loc[self.prev_date, 'CostOfRevenue']) / self.stock.income_statement.loc[self.prev_date, 'TotalRevenue']
#         return 1 if gross_margin_now > gross_margin_prev else 0

#     def calculate_asset_turnover_score(self):
#         asset_turnover_now = self.stock.income_statement.loc[self.date, 'TotalRevenue'] / self.stock.balance_sheet.loc[self.date, 'TotalAssets']
#         asset_turnover_prev = self.stock.income_statement.loc[self.prev_date,
#         'TotalRevenue'] / self.stock.balance_sheet.loc[self.prev_date, 'TotalAssets']
#         return 1 if asset_turnover_now > asset_turnover_prev else 0

#     def calculate_score(self):
#         self.stock.fetch_data()
#         score_data = []

#         for date in self.stock.balance_sheet.index[1:]:
#             self.set_dates(date)

#             roa_score = self.calculate_roa_score()
#             cfo_score = self.calculate_cfo_score()
#             delta_roa_score = self.calculate_delta_roa_score()
#             quality_of_earnings_score = self.calculate_quality_of_earnings_score()
#             delta_leverage_score = self.calculate_delta_leverage_score()
#             delta_liquidity_score = self.calculate_delta_liquidity_score()
#             new_equity_score = self.calculate_new_equity_score()
#             gross_margin_score = self.calculate_gross_margin_score()
#             asset_turnover_score = self.calculate_asset_turnover_score()

#             total_score = sum([roa_score, cfo_score, delta_roa_score, quality_of_earnings_score, delta_leverage_score,
#                                 delta_liquidity_score, new_equity_score, gross_margin_score, asset_turnover_score])
            
#             # Save the data
#             score_data.append([date, roa_score, cfo_score, delta_roa_score, quality_of_earnings_score, 
#                                delta_leverage_score, delta_liquidity_score, new_equity_score, 
#                                gross_margin_score, asset_turnover_score, total_score])

#         # Convert the data into a DataFrame
#         scores_df = pd.DataFrame(score_data, columns=['Date', 'ROA', 'CFO', 'Delta ROA', 'Quality of Earnings', 
#                                                       'Delta Leverage', 'Delta Liquidity', 'New Equity', 
#                                                       'Gross Margin', 'Asset Turnover', 'Piotroski Score'])
#         scores_df.set_index('Date', inplace=True)

#         return scores_df


In [6]:

# Use the classes
stocks = ['AAPL', 'MSFT', 'GOOGL']  # Add any stocks you're interested in
all_scores = []

for symbol in stocks:
    try:
        stock = utils.StockFundamentals(symbol)
        calculator = utils.PiotroskiScoreCalculator(stock)
        scores = calculator.calculate_score()
        scores['Ticker'] = symbol  # add a column for the ticker
        all_scores.append(scores)
    except Exception as e:
        print(f'Couldnt fetch data for {symbol} with error {e}')
        continue

# Concatenate all the individual DataFrames together
all_scores_df = pd.concat(all_scores)

all_scores_df

,ROA,CFO,Delta ROA,Quality of Earnings,Delta Leverage,Delta Liquidity,New Equity,Gross Margin,Asset Turnover,Piotroski Score,Ticker
Date,,,,,,,,,,,
2022-06-30,1,1,0,1,1,0,0,0,0,4,AAPL
2022-09-30,1,1,1,1,1,1,0,0,1,7,AAPL
2022-12-31,1,1,1,1,0,1,0,1,1,7,AAPL
2023-03-31,1,1,0,1,0,1,0,1,0,5,AAPL
2022-06-30,1,1,0,1,1,0,0,0,0,4,MSFT
2022-09-30,1,1,1,1,1,1,0,1,0,7,MSFT
2022-12-31,1,1,0,0,1,1,0,0,1,5,MSFT
2023-03-31,1,1,1,1,1,0,0,1,0,6,MSFT
2022-06-30,1,1,0,1,0,0,0,1,1,5,GOOGL


In [7]:
stock.income_statement

,periodType,currencyCode,AverageDilutionEarnings,BasicAverageShares,BasicEPS,CostOfRevenue,DilutedAverageShares,DilutedEPS,DilutedNIAvailtoComStockholders,EBIT,...,SellingAndMarketingExpense,SellingGeneralAndAdministration,TaxEffectOfUnusualItems,TaxProvision,TaxRateForCalcs,TotalExpenses,TotalOperatingIncomeAsReported,TotalRevenue,TotalUnusualItems,TotalUnusualItemsExcludingGoodwill
asOfDate,,,,,,,,,,,,,,,,,,,,,
2022-03-31,3M,USD,NaN,1.320341e+10,1.245001,2.959900e+10,1.335101e+10,1.231001,1.643600e+10,1.901700e+10,...,5.825000e+09,9.199000e+09,-1.992173e+08,2.498000e+09,0.131932,4.791700e+10,2.009400e+10,6.801100e+10,-1.510000e+09,-1.510000e+09
2022-06-30,3M,USD,1.072000e+09,1.313299e+10,1.220001,3.010400e+10,1.323899e+10,1.210001,1.600200e+10,1.909700e+10,...,6.630000e+09,1.028700e+10,-2.060909e+08,3.012000e+09,0.158410,5.023200e+10,1.945300e+10,6.968500e+10,-1.301000e+09,-1.301000e+09
2022-09-30,3M,USD,NaN,1.301800e+10,1.070000,3.115800e+10,1.309700e+10,1.060000,1.391000e+10,1.633400e+10,...,6.929000e+09,1.052600e+10,-2.166588e+08,2.323000e+09,0.143104,5.195700e+10,1.713500e+10,6.909200e+10,-1.514000e+09,-1.514000e+09
2022-12-31,3M,USD,NaN,1.289700e+10,1.060000,3.534200e+10,1.294700e+10,1.050000,1.362400e+10,1.723700e+10,...,7.183000e+09,1.227900e+10,-3.796876e+08,3.523000e+09,0.205459,5.788800e+10,1.816000e+10,7.604800e+10,-1.848000e+09,-1.848000e+09
2023-03-31,3M,USD,NaN,1.278100e+10,1.180000,3.061200e+10,1.282300e+10,1.170000,1.505100e+10,1.828500e+10,...,6.533000e+09,1.029200e+10,-2.182939e+07,3.154000e+09,0.173249,5.237200e+10,1.741500e+10,6.978700e+10,-1.260000e+08,-1.260000e+08


In [ ]:
scores_df = pd.DataFrame(scores)

# Get the last valid score and date for each ticker
latest_scores = {ticker: {"Date": scores_df[ticker].last_valid_index(), "Score": scores_df[ticker][scores_df[ticker].last_valid_index()]} for ticker in scores_df.columns}

# Convert to a DataFrame
latest_scores_df = pd.DataFrame(latest_scores).T

latest_scores_df


In [ ]:
# Only keep the 'GICS Sector' column
df_ticker_sector = dim_company[['Security', 'GICS Sector']]

# Join the two dataframes
merged_df = df_ticker_sector.join(latest_scores_df)

# Sort by 'GICS Sector' and 'Score'
merged_df.sort_values(by=['GICS Sector', 'Score'], ascending=[True, False], inplace=True)

merged_df


In [ ]:
# Only keep the 'GICS Sector' column
df_ticker_sector = dim_company[['Security', 'GICS Sub-Industry']]

# Join the two dataframes
merged_df = df_ticker_sector.join(latest_scores_df)

# Sort by 'GICS Sector' and 'Score'
merged_df.sort_values(by=['GICS Sub-Industry', 'Score'], ascending=[True, False], inplace=True)

merged_df
